# Exploring Singapore's food culture near major train stations

## Introduction

**Background**

Singapore's Mass Rapid Transport (MRT) is well connected is one of the main transport mode in the country. In 2019, the average daily ridership is at 3.384 million, which is about half the population of Singapore. 

Like any country, going out for food and drinks is common and is a part of the culture where people spending time catching-up with each other, or just trying out new places. Similarly, businesses are also changing, and new ones are appearing. 

**Problem area**

Singapore is a small country and business competition is high. For both business owners and customers, there are too many options to choose from, or compete against.

For this project, we will try to determine the top 10 food and drinks places located around major MRT interchanges. 

**Target Audience**

Hopefully, this observation can help new business owners understand the landscape just a little more before they startup, as well as to help customers  pick their next outing. 

**Scope**

For the purpose of this project, MRT interchanges are the main train networks that connects 2 or more other networks, and excludes those that connects to Light Rail Transits stations.

## Data

The datasets required for this analysis are:

1. List of MRT interchanges and its latitude and longitude. The source can be found in [data.world](https://data.world/hxchua/train-stations-in-singapore) and has the features required:
    1. stn_name
    2. stn_no
    3. latitude
    4. longitude
    5. color (color of the MRT network link)  
    
2. Data for busiest MRT interchanges. [mytransport.sg](http://mytransport.sg) provides monthly ridership statistics, however the data needs to be understood further before using, as it records the entry and exits of the station gantries instead of the passenger count.
3. Nearby venues from Foursquare. We will be using the `explore` API endpoint to source for venues near each of the MRT interchange

At this point, the following sources are identified:

- data.gov.sg
- mytransport.sg
- data.world
- Wikipedia
- Foursquare

## Methodology

### Import required libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import folium
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print ("Libraries loaded")

Libraries loaded


### 1. Identifying MRT Interchanges
We'll start by taking a look at the list of MRT stations taken from the data.world source. 

In [2]:
sg_mrt_stations_all = pd.read_csv( "data/dataworld-singapore-mrt-stations.csv" )

# remove columns we don't need
sg_mrt_stations_all.drop( ['OBJECTID', 'X', 'Y' ], axis=1, inplace=True )

# rename some columns
sg_mrt_stations_all.rename( columns={ 'STN_NAME': 'Station Name', 'STN_NO':'Station No', 'COLOR': 'Color' }, inplace=True)

# convert color values to lowercase
sg_mrt_stations_all['Color'] = sg_mrt_stations_all['Color'].str.lower()

sg_mrt_stations_all.head()


,Station Name,Station No,Latitude,Longitude,Color
0,ADMIRALTY MRT STATION,NS10,1.440585,103.800998,red
1,ALJUNIED MRT STATION,EW9,1.316433,103.882893,green
2,ANG MO KIO MRT STATION,NS16,1.369933,103.849553,red
3,BAKAU LRT STATION,SE3,1.388093,103.905418,others
4,BANGKIT LRT STATION,BP9,1.380018,103.772667,others


In [3]:
# check the shape
sg_mrt_stations_all.shape

(187, 5)

The data provided is structured in a way that the MRT station names that are interchanges are repeated for each train network it is connected to. We can quickly verify this:

In [4]:
# sg_mrt_stations_all["STN_NAME"].value_counts()
#In [36]: DataFrame({'count' : df1.groupby( [ "Name", "City"] ).size()}).reset_index()

sg_mrt_stations_all_grouped = sg_mrt_stations_all.loc[:].groupby('Station Name').count()
sg_mrt_stations_all_grouped.rename( columns = { 'Station No' : 'Total Stations' }, inplace=True)
sg_mrt_stations_all_grouped[ ['Total Stations'] ].sort_values( ['Total Stations'], ascending=False ).head(10)



,Total Stations
Station Name,
DHOBY GHAUT MRT STATION,3
BOTANIC GARDENS MRT STATION,2
SERANGOON MRT STATION,2
HARBOURFRONT MRT STATION,2
RAFFLES PLACE MRT STATION,2
CITY HALL MRT STATION,2
PROMENADE MRT STATION,2
CHINATOWN MRT STATION,2
TAMPINES MRT STATION,2


From the list above, we can check if **DHOBY GHAUT MRT STATION** is connected to **3** train network lines:

In [5]:
sg_mrt_stations_all[ sg_mrt_stations_all["Station Name"] == "DHOBY GHAUT MRT STATION" ]

,Station Name,Station No,Latitude,Longitude,Color
51,DHOBY GHAUT MRT STATION,NS24,1.298701,103.846112,red
52,DHOBY GHAUT MRT STATION,NE6,1.299705,103.845485,purple
53,DHOBY GHAUT MRT STATION,CC1,1.298843,103.846236,yellow


We can confirm that it is indeed connected to 3 stations; **NS24**, **NE6**, and **CC1**. With this, we will now proceed with further cleaning and drop rows that are not interchanges.

In [6]:

# First we'll create a dataframe that contains only NON interchanges
sg_mrt_stations_not_inx_names = sg_mrt_stations_all_grouped.loc[:].reset_index()
# sg_mrt_stations_not_inx_names.drop(['Latitude', 'Longitude', 'Color'], axis=1,  inplace=True )

# get the indexes that are indexes to drop
indexes_to_drop = sg_mrt_stations_not_inx_names[ sg_mrt_stations_not_inx_names['Total Stations'] > 1 ].index 

# drop
sg_mrt_stations_not_inx_names.drop(indexes_to_drop, inplace = True)

sg_mrt_stations_not_inx_names.reset_index(inplace=True, drop=True)

# sg_mrt_stations_not_inx_names


# get a copy of the full list
sg_mrt_stations_inx = sg_mrt_stations_all.loc[:]

# loop through the list of NON interchanges
for row in sg_mrt_stations_not_inx_names['Station Name']:
    # drop it
    sg_mrt_stations_inx.drop( sg_mrt_stations_inx[ sg_mrt_stations_inx['Station Name'] == row ].index, inplace = True )


sg_mrt_stations_inx.reset_index(inplace=True, drop=True)

sg_mrt_stations_inx.head(10)


,Station Name,Station No,Latitude,Longitude,Color
0,BAYFRONT MRT STATION,DT16,1.281874,103.859073,blue
1,BAYFRONT MRT STATION,CE1,1.281874,103.859073,others
2,BISHAN MRT STATION,CC15,1.351309,103.849150,yellow
3,BISHAN MRT STATION,NS17,1.350839,103.848140,red
4,BOTANIC GARDENS MRT STATION,DT9,1.322424,103.816136,blue
5,BOTANIC GARDENS MRT STATION,CC19,1.322110,103.814988,yellow
6,BUGIS MRT STATION,DT14,1.299551,103.856862,blue
7,BUGIS MRT STATION,EW12,1.300465,103.855701,green
8,BUONA VISTA MRT STATION,EW21,1.307183,103.790203,green
9,BUONA VISTA MRT STATION,CC22,1.306492,103.790665,yellow


The provided data contains a **'color'** column which corresponds to official color (not in exact color code) assigned to the train network line. We note that there are colors assigned as 'Others':

In [7]:
sg_mrt_stations_inx[ sg_mrt_stations_inx['Color'] == 'others' ]

,Station Name,Station No,Latitude,Longitude,Color
1,BAYFRONT MRT STATION,CE1,1.281874,103.859073,others
17,EXPO MRT STATION,CG1,1.334550,103.961513,others
28,MARINA BAY MRT STATION,CE2,1.276151,103.855089,others
44,WOODLANDS MRT STATION,TE2,1.436067,103.787945,others


We will correct these values manually as it is faster that to do so. Color references can be checked here: https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations#MRT_stations

In [8]:
# Update using the indexes from the table above
sg_mrt_stations_inx.loc[1, 'Color'] = 'yellow'
sg_mrt_stations_inx.loc[17, 'Color'] = 'green'
sg_mrt_stations_inx.loc[28, 'Color'] = 'yellow'
sg_mrt_stations_inx.loc[44, 'Color'] = 'brown'


# let's check the values again
sg_mrt_stations_inx.loc[[1, 17, 28, 44]]

,Station Name,Station No,Latitude,Longitude,Color
1,BAYFRONT MRT STATION,CE1,1.281874,103.859073,yellow
17,EXPO MRT STATION,CG1,1.334550,103.961513,green
28,MARINA BAY MRT STATION,CE2,1.276151,103.855089,yellow
44,WOODLANDS MRT STATION,TE2,1.436067,103.787945,brown


Now we will plot these on a map to visulize their positions.

In [9]:
address = 'Singapore, SG'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore is 1.3408630000000001, 103.83039182212079.


In [10]:
map = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='Stamen Terrain')


# add markers to map
for lat, lng, label, color in zip(sg_mrt_stations_inx['Latitude'], sg_mrt_stations_inx['Longitude'], sg_mrt_stations_inx['Station Name'] + "(" + sg_mrt_stations_inx['Station No'] + ")", sg_mrt_stations_inx['Color'] ):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
    
map

We have now plotted the stations on each interchange, however, we are more concerned with the general surrounding of the interchange. Let's clean up the data little further but grouping the data by interchange name and using the `mean` for its latitude and longitude

In [11]:
    
mean_coords = sg_mrt_stations_inx.loc[:]
mean_coords.drop(['Station No', 'Color'], axis=1, inplace=True )
mean_coords = mean_coords.groupby('Station Name').mean()
mean_coords.reset_index(inplace=True)
# print(mean_coords.shape)
# print( mean_coords[ mean_coords['Station Name'] == 'BISHAN MRT STATION'] )

current_station = ''

sg_mrt_inx_merged = pd.DataFrame(columns=['Station Name', 'Station Nos', 'Colors', 'Latitude', 'Longitude'] )

for index, row in sg_mrt_stations_inx.iterrows():
    
    if( current_station != row['Station Name'] ):
        
        if(current_station != ''):
            sg_mrt_inx_merged = sg_mrt_inx_merged.append({
                'Station Name': current_station, 
                'Station Nos': ','.join(station_nos),
                'Colors': ','.join(colors),
                'Latitude': mean_coords[ mean_coords['Station Name'] == current_station]['Latitude'].iloc[0],
                'Longitude': mean_coords[ mean_coords['Station Name'] == current_station]['Longitude'].iloc[0]
            }, ignore_index=True)
        
        colors = []
        station_nos = []
        current_station = row['Station Name']
#         print(current_station)
        
    colors.append(row['Color'])
    station_nos.append(row['Station No'])

# save the last data    
sg_mrt_inx_merged = sg_mrt_inx_merged.append({
    'Station Name': current_station, 
    'Station Nos': ','.join(station_nos),
    'Colors': ','.join(colors),
    'Latitude': mean_coords[ mean_coords['Station Name'] == current_station]['Latitude'].iloc[0],
    'Longitude': mean_coords[ mean_coords['Station Name'] == current_station]['Longitude'].iloc[0]
}, ignore_index=True)

# check that we have the same number of rows
mean_coords.shape[0] == sg_mrt_inx_merged.shape[0]
    

True

In [50]:
sg_mrt_inx_merged.head()

,Station Name,Station Nos,Colors,Latitude,Longitude
0,BAYFRONT MRT STATION,"DT16,CE1","blue,yellow",1.281874,103.859073
1,BISHAN MRT STATION,"CC15,NS17","yellow,red",1.351074,103.848645
2,BOTANIC GARDENS MRT STATION,"DT9,CC19","blue,yellow",1.322267,103.815562
3,BUGIS MRT STATION,"DT14,EW12","blue,green",1.300008,103.856281
4,BUONA VISTA MRT STATION,"EW21,CC22","green,yellow",1.306838,103.790434


In [42]:
map = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='Stamen Terrain')


# add markers to map
for lat, lng, label, color in zip(sg_mrt_inx_merged['Latitude'], sg_mrt_inx_merged['Longitude'], sg_mrt_inx_merged['Station Name'] + " (" + sg_mrt_inx_merged['Station Nos'] + ")", sg_mrt_inx_merged['Colors'] ):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
    
map

## 2. Exploring the busiest interchanges

### TO ADD

In [33]:
passengers_oct2020 = pd.read_csv('data/transport_node_train_202010.csv')

  
Lets examine the data

In [34]:
passengers_oct2020.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2020-10,WEEKDAY,11,TRAIN,NS7,2353,1912
1,2020-10,WEEKENDS/HOLIDAY,11,TRAIN,NS7,1434,1940
2,2020-10,WEEKDAY,16,TRAIN,SW4,1033,1457
3,2020-10,WEEKENDS/HOLIDAY,16,TRAIN,SW4,514,522
4,2020-10,WEEKDAY,10,TRAIN,CC5,1319,3515


The data shows the hourly (TIME_PER_HOUR) taps in and out of each station, separated by Weekdays and Weekends/Holiday. We will clean up the data now by doing a few things:
- Create a new AVG_PASSENGERS column as a mean of the Tap In and Tap Out values
- Drop the Tap In and Tap Out columns after calculating the mean
- Drop the PT_TYPE column as we know that this data is for TRAIN only. 

In [35]:
passengers_oct2020['AVG_PASSENGERS'] = passengers_oct2020[['TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME'] ] .mean(axis=1)

# remove columns we don't need
passengers_oct2020.drop( ['PT_TYPE', 'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME'], axis=1, inplace=True )

In [36]:
passengers_oct2020.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_CODE,AVG_PASSENGERS
0,2020-10,WEEKDAY,11,NS7,2132.5
1,2020-10,WEEKENDS/HOLIDAY,11,NS7,1687.0
2,2020-10,WEEKDAY,16,SW4,1245.0
3,2020-10,WEEKENDS/HOLIDAY,16,SW4,518.0
4,2020-10,WEEKDAY,10,CC5,2417.0


In [39]:
# passengers_oct2020[ passengers_oct2020['PT_CODE'] == sg_mrt_stations_inx['Station No'] ]
passengers_oct2020['PT_CODE']

0                NS7
1                NS7
2                SW4
3                SW4
4                CC5
5                CC5
6               CC23
7               CC23
8               EW33
9               EW33
10              CC21
11              CC21
12          EW24/NS1
13          EW24/NS1
14              NE10
15              NE10
16              EW22
17              EW22
18              CC16
19              CC16
20              EW11
21              EW11
22      NS24/NE6/CC1
23      NS24/NE6/CC1
24              EW19
25              EW19
26         CC22/EW21
27         CC22/EW21
28               CC8
29               CC8
30          NE7/DT12
31          NE7/DT12
32              EW33
33              EW33
34              NS28
35              NS28
36              DT10
37              DT10
38          EW16/NE3
39          EW16/NE3
40              CC14
41              CC14
42              EW27
43              EW27
44          NS27/CE2
45          NS27/CE2
46              DT27
47           

## Results

## Discussion

## Conclusion

In [13]:
# import pandas library 
import pandas as pd 
  
# dictionary with list object in values 
details = { 
    'Name' : ['Ankit', 'Aishwarya', 'Shaurya', 
              'Shivangi', 'Priya', 'Swapnil'], 
    'Age' : [23, 21, 22, 21, 24, 25], 
    'University' : ['BHU', 'JNU', 'DU', 'BHU',  
                    'Geu', 'Geu'], 
} 
  
# creating a Dataframe object  
df = pd.DataFrame(details, columns = ['Name', 'Age', 
                                      'University'], 
                  index = ['a', 'b', 'c', 'd', 'e', 'f']) 
  
# get names of indexes for which 
# column Age has value 21 
index_names = df[ df['Age'] == 21 ].index 
  
# # drop these row indexes 
# # from dataFrame 
# df.drop(index_names, inplace = True) 
  
# df 
index_names

Index(['b', 'd'], dtype='object')